In [36]:
import pybamm
import pybammeis
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy

In [ ]:
def findV(steps):
# Find the discharge step
    for step in steps:
        if "Discharge" in step and "until" in step and "V" in step:
            # Split by 'until' and then extract the number before 'V'
            parts = step.split("until")
            if len(parts) > 1:
                voltage_part = parts[1].strip()  # e.g., "2.5 V"
                voltage_value = float(voltage_part.replace("V", "").strip())
                print("Voltage limit:", voltage_value)
                return voltage_value


# Define a class containing the DFN model or PMe model with degradation mechanisms(SEI growth & lithium plating and particle cracking)
class simul():
    def __init__(self, profile, cycles):
        self.exp = profile

        self.model = pybamm.lithium_ion.DFN(
            {
            "thermal": "isothermal",
            "SEI": "solvent-diffusion limited",
            "SEI porosity change": "true",
            "lithium plating": "partially reversible",
            "lithium plating porosity change": "true",  # alias for "SEI porosity change"
            "particle mechanics": ("swelling and cracking","swelling only"),
            "SEI on cracks": "true",
                "loss of active material": "stress-driven",
                "calculate discharge energy": "true",  # for compatibility with older PyBaMM versionsg
            })
        
        self.aged_model = deepcopy(self.model)

        self.param = pybamm.ParameterValues("OKane2022")
        self.param.update({
            "Initial temperature [K]": 278.15, #change the temperature 
            "SEI growth activation energy [J.mol-1]": 37000.0,
            "Ambient temperature [K]": 293.15,},)

        # Discretization points
        self.var_pts = {
            "x_n": 5,
            "x_s": 5,
            "x_p": 5,
            "r_n": 8,
            "r_p": 8,
        }
        self.solver = pybamm.IDAKLUSolver(atol=1e-5, rtol=1e-5)
        self.cycle_number = cycles  # for  aging

        self.data = {} #filled after the simulation, used to evaluate fitness

    def run(self):
        solutions = []
        # Run 1 full charge/discharge/hold cycle at a time
        for cycle in range(1, self.cycle_number + 1):
            print(f"\n Running cycle {cycle}...")

            # Run the cycle using the aged model 
            sim = pybamm.Simulation(
                self.aged_model,
                parameter_values=self.param,
                experiment=self.exp,
                solver=self.solver,
                var_pts=self.var_pts,
            )
            sol = sim.solve(calc_esoh=True,initial_soc=1)
            if cycle in [1, self.cycle_number]:
                solutions.append(sol)
        
        # Extract relevant info to later evaluate the model
        SOH0 = solutions[1].summary_variables["Capacity [A.h]"][0] / solutions[0].summary_variables["Capacity [A.h]"][0]
        SOH1 = solutions[1].summary_variables["Capacity [A.h]"][-1] / solutions[0].summary_variables["Capacity [A.h]"][-1]
        data['delta SOH'] = SOH0 - SOH1

        # Now let's get the time elapsed for a complete charge
        # Consider only the first one, greedy approach that mirrors the request
        self.data['time'] = solutions[0]["Time [s]"].entries[-1] - solutions[0]["Time [s]"].entries[0]

        # Compute energy loss during the charging process
        # find where the discharging step ends
        my_v = findV(self.exp.cycles[0])
        voltage = solutions[0]["Voltage [V]"].entries
        lower_cut = np.where(voltage <= my_v)[0][0]  # Find the first index where voltage is less or equal to my_v
        voltage = voltage[lower_cut:]
        # Find the index where the voltage stops growing
        voltage_diff = np.diff(voltage, prepend=voltage[0])
        upper_cut = np.where(voltage_diff <= 0)[0][0]  # First index where voltage stops increasing
        voltage = voltage[:upper_cut]
        time = solutions[0]["Time [s]"].entries[lower_cut:upper_cut]
        current = solutions[0]["Current [A]"].entries[lower_cut:upper_cut]

        # Compute energy input using numpy dot product
        print(f'{voltage}\n {time}\n{current}')
        energy_input = np.dot(voltage, np.diff(time, prepend=0) * current) / 3600  # Convert to Wh

        # Calculate energy output
        final_capacity = solutions[0].summary_variables["Capacity [A.h]"][-1]
        final_voltage = voltage[-1]
        energy_output = final_capacity * final_voltage  # Energy stored in the battery (Wh)

        # Calculate the energy loss
        print(f'{energy_input} , {energy_output}')
        energy_loss = energy_input - energy_output

        # Store energy loss and efficiency in the data dictionary
        self.data['energy_loss'] = round(energy_loss, 4)
        self.data['efficiency'] = (energy_output / energy_input) * 100  # Charging efficiency in percentage

    def evaluate_fitness(profile=None):
        if profile.lower == 'fast charge':
            return

        if profile == 'sustainable':
            return

        if profile == 'long life':
            return 

        # Here profile can only be 'all day'
        return

In [ ]:
# Define what is needed to do genetic algorithm to find best profiles
es = pybamm.Experiment([
    (
        "Discharge at 1C until 2.5 V",
        "Charge at 5 A until 4.2 V",
        "Hold at 4.2 V until C/100",
        "Rest for 30 minutes",
    )
],period="30 minutes",) #change this for good accuracy

test = simul(es, 4)
test.run()


 Running cycle 1...

 Running cycle 2...

 Running cycle 3...

 Running cycle 4...
-1.8487580980601592 , 21.393818776158426
{'SOH': 1.0, 'time': 12044.890264810203, 'energy_loss': -23.2426, 'efficiency': -1157.1994626342005}


IndexError: index -1 is out of bounds for axis 0 with size 0

In [34]:
profiles = ['fast charge', 'sustainable', 'long life', 'all day']
from math import sqrt

# Genetic stuff
def random_profile():

    a, b = 2, 5  # number of charges setted with a beta
    number_of_charge = int(max(1, np.ceil(6 * np.random.beta(a, b))))

    ampere_per_charge = [max(0.5, min(np.random.normal(0.5, 0.5), 2))]
    for i in range (1, number_of_charge):
        ampere_per_charge.append(max(ampere_per_charge[i-1] + 0.1, min(np.random.normal(ampere_per_charge[i-1], 1), 2)))
    
    time_per_charge = [int(max(15, min(np.random.normal(20, 10), 30))) for _ in range (number_of_charge)] #if the number of charge are 1, no time is used but only the until 4.2 V
    
    hold_cutoff = np.random.choice(["C/100", "C/50", "C/20"])

    sol = ["Discharge at 1C until 2.5 V"]

    for i in range (number_of_charge):
        if i != number_of_charge - 1:
            sol.append(f"Charge at {ampere_per_charge[i]:.2f} A for {time_per_charge[i]} minutes")
            if np.random.uniform(0, 1) < 0.2: # 20% possibility of a little rest
                sol.append(f"Rest for {int(max(10, min(np.random.normal(20, 10), 30)))} minutes")
        else:
            s = f"Charge at {ampere_per_charge[i]:.2f} A until 4.2 V"
            sol.append(s)
            sol.append(f"Hold at 4.2 V until {hold_cutoff}")
            sol.append("Rest for 30 minutes")

    return sol

print(random_profile())

['Discharge at 1C until 2.5 V', 'Charge at 0.50 A for 15 minutes', 'Rest for 10 minutes', 'Charge at 0.70 A until 4.2 V', 'Hold at 4.2 V until C/50', 'Rest for 30 minutes']


In [40]:
es = pybamm.Experiment([
    tuple(random_profile())
],period="30 minutes",) #change this for good accuracy

test = simul(es, 4)
test.run()


 Running cycle 1...

 Running cycle 2...

 Running cycle 3...

 Running cycle 4...
-2.8724698849034067 , 21.31674390069497


In [42]:
print(test.data)

{'SOH': 1.0, 'time': 15531.707904690102, 'energy_loss': -24.1892, 'efficiency': -742.1050439110797}
